In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GCode.Line
import GRBL


# Code:

Draw a 10 mm line from (0, 0) to (10, 0).

In [2]:
progs = list()

for n_points in range(2, 10):
    
    line_cfg = {
        "X0": 0,
        "Xf": 10,
        "Y": 0,
        "n_points": n_points
    }    
    points = GCode.HorzLine(**line_cfg)
    
    line_cfg = {
        "points": points,
        "feed":120,
        "power":128,
        "dynamic_power": True,
    }
    line = GCode.Line(points=points)
    
    
    prog_cfg={
        "lines": [line, line],
        "feed": 120
    }
    prog = GCode.Program(**prog_cfg)
    progs.append(prog)
progs

[Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>]

In [3]:
import threading

def concurrent_map(func, data):
    """
    Similar to the bultin function map(). But spawn a thread for each argument
    and apply `func` concurrently.

    Note: unlike map(), we cannot take an iterable argument. `data` should be an
    indexable sequence.
    """

    N = len(data)
    result = [None] * N

    # wrapper to dispose the result in the right slot
    def task_wrapper(i):
        result[i] = func(data[i])

    threads = [threading.Thread(target=task_wrapper, args=(i,)) for i in range(N)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    return result

In [4]:
concurrent_map(lambda prog: prog.generate_gcode(), progs)

[None, None, None, None, None, None, None, None]

In [5]:
concurrent_map(lambda prog: len(prog.buffer), progs)

[17, 19, 21, 23, 25, 27, 29, 31]

In [6]:
concurrent_map(lambda prog: prog.__repr__(), progs)

['Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>']

In [7]:
concurrent_map(lambda prog: prog.dist, progs)

[40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0]

In [8]:
concurrent_map(lambda prog: prog.jog_dist, progs)

[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]

In [9]:
concurrent_map(lambda prog: prog.laserin_dist, progs)

[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]

In [11]:
for prog in progs:
    for line in prog.lines:
        break
    break

In [12]:
prog

Program<lines=2, dist=40.00mm, t=14.00s>

In [13]:
line

Line<len=10.0mm, feed=300, power=150>

In [14]:
line.reverse()
line

Line<len=10.0mm, feed=300, power=150>

In [15]:
line.generate_gcode()
line

Line<len=10.0mm, feed=300, power=150>